## 工具调用

In [1]:
# How to use chat models to call tools

# The function name, type hints, and docstring are all part of the tool
# schema that's passed to the model. Defining good, descriptive schemas
# is an extension of prompt engineering and is an important part of
# getting models to perform well.
def add(a: int, b: int) -> int:
    """Add two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a + b


def multiply(a: int, b: int) -> int:
    """Multiply two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a * b

In [2]:
from pydantic import BaseModel, Field


class add(BaseModel):
    """Add two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


class multiply(BaseModel):
    """Multiply two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")

In [3]:
from typing_extensions import Annotated, TypedDict


class add(TypedDict):
    """Add two integers."""

    # Annotations must have the type and can optionally include a default value and description (in that order).
    a: Annotated[int, ..., "First integer"]
    b: Annotated[int, ..., "Second integer"]


class multiply(TypedDict):
    """Multiply two integers."""

    a: Annotated[int, ..., "First integer"]
    b: Annotated[int, ..., "Second integer"]


tools = [add, multiply]

In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="Qwen/Qwen3-8B",
    # model="Qwen/Qwen3-30B-A3B-Instruct-2507",
    api_key="sk-jvjyawqpodlkxlywatvemcdykkrbvthhjyjyapyvtnifwlbl",
    base_url="https://api.siliconflow.cn/v1/",
    # temperature=0,
    )

llm_with_tools = llm.bind_tools(tools)

query = "What is 3 * 12?"

llm_with_tools.invoke(query)

AIMessage(content='\n\n', additional_kwargs={'tool_calls': [{'id': '0198d185f14764d3e064c094c4eae946', 'function': {'arguments': ' {"a": 3, "b": 12}', 'name': 'multiply'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 123, 'prompt_tokens': 265, 'total_tokens': 388, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 108, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': None}, 'model_name': 'Qwen/Qwen3-8B', 'system_fingerprint': '', 'id': '0198d185de45f336b8ba1e358e810bad', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--8c462a38-1f47-4e74-88b6-eeb9f55b7b2b-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': '0198d185f14764d3e064c094c4eae946', 'type': 'tool_call'}], usage_metadata={'input_tokens': 265, 'output_tokens': 123, 'total_tokens': 388, 'input_token_details': {}, 'output_token_details': {'reasoning':

In [5]:
# Tool calls
query = "What is 3 * 12? Also, what is 11 + 49?"

llm_with_tools.invoke(query).tool_calls


[{'name': 'multiply',
  'args': {'a': 3, 'b': 12},
  'id': '0198d1870da05135699437527de9309b',
  'type': 'tool_call'},
 {'name': 'add',
  'args': {'a': 11, 'b': 49},
  'id': '0198d187128c3fe67647c76a3ece63ec',
  'type': 'tool_call'}]

In [6]:
# Parsing
from langchain_core.output_parsers import PydanticToolsParser
from pydantic import BaseModel, Field


class add(BaseModel):
    """Add two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


class multiply(BaseModel):
    """Multiply two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


chain = llm_with_tools | PydanticToolsParser(tools=[add, multiply])
chain.invoke(query)

[multiply(a=3, b=12), add(a=11, b=49)]